# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from IPython.display import HTML

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
print(data.info())
HTML(data.head().to_html()) #prints first five lines without truncating columns

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,occupbroad,workinschool,email,computerskills,specialskills,firstname,sex,race,h,l,call,city,kind,adid,fracblack,fracwhite,lmedhhinc,fracdropout,fraccolp,linc,col,expminreq,schoolreq,eoe,parent_sales,parent_emp,branch_sales,branch_emp,fed,fracblack_empzip,fracwhite_empzip,lmedhhinc_empzip,fracdropout_empzip,fraccolp_empzip,linc_empzip,manager,supervisor,secretary,offsupport,salesrep,retailsales,req,expreq,comreq,educreq,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,1,0,0,1,0,Allison,f,w,0.0,1.0,0.0,c,a,384.0,0.989360,0.005500,9.527484,0.274151,0.037662,8.706325,1.0,5,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,6,1,1,1,0,Kristen,f,w,1.0,0.0,0.0,c,a,384.0,0.080736,0.888374,10.408828,0.233687,0.087285,9.532859,0.0,5,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,1,1,0,1,0,Lakisha,f,b,0.0,1.0,0.0,c,a,384.0,0.104301,0.837370,10.466754,0.101335,0.591695,10.540329,1.0,5,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,5,0,1,1,1,Latonya,f,b,1.0,0.0,0.0,c,a,384.0,0.336165,0.637370,10.431908,0.108848,0.406576,10.412141,0.0,5,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,5,1,1,1,0,Carrie,f,w,1.0,0.0,0.0,c,a,385.0,0.397595,0.180196,9.876219,0.312873,0.030847,8.728264,0.0,some,,1.0,9.4,143.0,9.4,143.0,0.0,0.204764,0.727046,10.619399,0.070493,0.369903,10.007352,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [4]:
# determine the number of successes and failures in both white and black groups
w = data[data.race=='w']
b = data[data.race=='b']

white_success = w.call.sum()
white_fail = w.call.count() - white_success

black_success = b.call.sum()
black_fail = b.call.count() - black_success

print(white_fail, white_success, black_fail, black_success)

2200.0 235.0 2278.0 157.0


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

The appropriate test here is a two-sample z-test to compare the proportion of callbacks in the black-name and white-name groups. 

The central limit theorem requires a large enough sample size, and independence of observations. As shown in the cell above, there are more than 10 successes and 10 failures for both the black and white group, so the sample size is large enough. The samples can also be assumed to be independent because the response to one resume should not be affected by the responses to any of the other resumes.

The null hypothesis is that resumes with white-sounding and black-sounding names have the same callback rate (the difference of means is equal to 0), while the alternative hypothesis is that the callback rates for the two kinds of resumes are different (the difference of means is not equal to 0).

In [5]:
# Your solution to Q3 here

In [6]:
# view callback rates for the two groups
w_call_rate = w.call.mean()
b_call_rate = b.call.mean()

print('white callback rate: ',w_call_rate, '\nblack callback rate: ', b_call_rate)

white callback rate:  0.09650924056768417 
black callback rate:  0.0644763857126236


In [7]:
# perform z-test for proportions
from statsmodels.stats.proportion import proportions_ztest

successes = [white_success, black_success]
nobs = [len(w.call), len(b.call)]

zstat, p_value = proportions_ztest(successes, nobs, value=0)

print('Z-test for proportions:')
print('z-statistic: ', zstat)
print('p-value: ', p_value)

# compute 95% confidence interval using z-score
w_var = w_call_rate * (1-w_call_rate)/len(w.call)
b_var = b_call_rate * (1-b_call_rate)/len(b.call)

sem_diff = (w_var + b_var)**0.5 #standard error for the difference between call rates
mean_diff = w_call_rate - b_call_rate
zscore = stats.norm.ppf(0.975)
conf_int_z = [mean_diff - zscore * sem_diff, mean_diff + zscore * sem_diff]
moe_z = (conf_int_z[1] - conf_int_z[0])/2

print('95% confidence interval: ', conf_int_z)
print('margin of error: ', moe_z)

Z-test for proportions:
z-statistic:  4.108412152434346
p-value:  3.983886837585077e-05
95% confidence interval:  [0.01677772882804784, 0.04728798088207331]
margin of error:  0.015255126027012735


In [8]:
# perform bootstrap sampling to compare proportions
# convert callback data to fit assumption that the two groups have the same callback rate
w_convert = w[['race', 'call']].copy()
b_convert = b[['race', 'call']].copy()

w_convert.call = w_convert.call - mean_diff/2
b_convert.call = b_convert.call + mean_diff/2

data_convert = pd.concat([w_convert, b_convert])
n_obs = len(data_convert.call)

n_samps = 10000
mean_diffs_bs = np.empty(n_samps)

for i in range(n_samps):
    bs_sample = data_convert.sample(n=n_obs, replace=True)
    w_sample = bs_sample[bs_sample.race == 'w']
    b_sample = bs_sample[bs_sample.race == 'b']
    
    mean_diffs_bs[i] = w_sample.call.mean() - b_sample.call.mean()

In [9]:
p_value = np.sum(mean_diffs_bs[mean_diffs_bs >= mean_diff])/len(mean_diffs_bs)

conf_int_bs = np.percentile(mean_diffs_bs, [2.5, 97.5]) + mean_diff
moe_bs = (conf_int_bs[1] - conf_int_bs[0])/2

print('Bootstrap sampling approach:')
print('95% confidence interval: ', conf_int_bs)
print('margin of error: ', moe_bs)
print('p-value: ', p_value)

Bootstrap sampling approach:
95% confidence interval:  [0.01699906 0.04722371]
margin of error:  0.015112328436225652
p-value:  0.0


___
Bootstrap sampling and z-testing for proportions agree that we can discard the null hypothesis, with both returning a p-value < 0.0001.
___

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

## Story
This statistical analysis shows that the callback rate is different for resumes with black-sounding names compared to those with white-sounding names. The callback rates seen in this data suggest that whites find greater success in getting responses from resume submissions, even when resumes are identical. This is strong evidence for the existence of racial discrimination in the hiring process.

We cannot conclude from this analysis, however, that race is the most important factor in getting responses from potential employers, or even that it is one of the most important. This analysis only sought to determine whether race had a statistically-significant impact. In order to determine the relative importance of race to other aspects of the resumes, we would have to determine which other fields in the dataset have a significant impact on callback rate, and compare the effect size of race to the effect sizes of those fields.